In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

In [ ]:
# Load the Stanford Online Products Dataset
dataset_name = 'stanford_online_products'
(train_dataset, test_dataset), dataset_info = tfds.load(
    name=dataset_name,
    split=['train', 'test'],
    with_info=True,
    as_supervised=False
)

In [ ]:
# Preprocess and prepare the data
def preprocess_data(data):
    image = data['image']
    label = data['class_id']
    # Preprocess the image (e.g., resizing, normalizing)
    image = tf.image.resize(image, (64, 64))  # Resize to a common size
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
    return image, label

train_dataset = train_dataset.map(preprocess_data)
test_dataset = test_dataset.map(preprocess_data)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(dataset_info.features['class_id'].num_classes, activation='softmax')
])

In [ ]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Set batch size and number of epochs
batch_size = 32
epochs = 50

In [ ]:
# Shuffle and batch the data
train_dataset = train_dataset.shuffle(1000).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [ ]:
# Train the model
model.fit(train_dataset, epochs=epochs)


Epoch 1/50
1861/1861 [==============================] - 68s 35ms/step - loss: 9.6516 - accuracy: 8.3962e-05
Epoch 2/50
1861/1861 [==============================] - 68s 36ms/step - loss: 8.8081 - accuracy: 0.0011
Epoch 3/50
1861/1861 [==============================] - 65s 34ms/step - loss: 8.0776 - accuracy: 0.0050
Epoch 4/50
1861/1861 [==============================] - 70s 37ms/step - loss: 7.2799 - accuracy: 0.0175
Epoch 5/50
1861/1861 [==============================] - 67s 35ms/step - loss: 6.3797 - accuracy: 0.0460
Epoch 6/50
1861/1861 [==============================] - 66s 35ms/step - loss: 5.4563 - accuracy: 0.1024
Epoch 7/50
1861/1861 [==============================] - 65s 35ms/step - loss: 4.6185 - accuracy: 0.1823
Epoch 8/50
1861/1861 [==============================] - 69s 37ms/step - loss: 3.8879 - accuracy: 0.2755
Epoch 9/50
1861/1861 [==============================] - 65s 34ms/step - loss: 3.2530 - accuracy: 0.3675
Epoch 10/50
1861/1861 [==============================] - 65s

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_accuracy:.2%}')

RuntimeError: ignored